In [1]:
import pandas as pd
import numpy as np
import glob
import os
import json
import uuid
import datetime
from time import sleep
import openai
import tiktoken
from blingfire import text_to_sentences
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm
tqdm.pandas()
import logging

c:\Users\sihao\Documents\projects\babel\venv_babel\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_paths():
    """
    Get paths to all documents in the database. Ignore paths which are directories
    Return: list of absolute file paths
    """
    doc_paths = [f for f in glob.glob(PATH_TO_DATABASE + '/**', recursive=True) if not os.path.isdir(f)]
    return [i.replace('\\', '/') for i in doc_paths]


In [3]:
def fragment_doc(doc):
    """
    Split a document into paragraphs of sentences.
    Returns a dict with info about the document, including the list of paragraphs.
    """
    paragraphs = []
    file_token_count = 0
    # use blingfire sentence boundary detection to split document into sentences
    sents = text_to_sentences(doc).split('\n')
    # combine sentences into paragraphs with overlap
    for i in range(0, len(sents), CHUNK_SIZE-CHUNK_OVERLAP):
        # using a slice, so even if [i:i+CHUNK_SIZE] is out of bounds, 
        # it will just return the remaining sentences
        # this is useful for the last paragraph, which may not have CHUNK_SIZE sentences
        para = ' '.join(sents[i:i+CHUNK_SIZE])
        # check if paragraph is too long for the embedding model
        file_num_tokens = len(enc.encode(para))
        if file_num_tokens > EMBEDDING_TOKEN_LIMIT:
            raise RuntimeError(f'{doc_name}: Paragraph too long: {para[:50]}...')
        # paragraph is short enough, so just add it to the list
        else:
            paragraphs.append(para)
        # update token count
        file_token_count += file_num_tokens
    # store info about the document in a dict
    doc_data = {'num_sents': len(sents),
                'num_paras': len(paragraphs),
                'num_tokens': file_token_count,
                'text': paragraphs}
    return doc_data


In [4]:
def read_documents():
    """
    Read all documents in the database into a list of dicts.
    Each dict contains info about the document, including the list of paragraphs.
    """
    doc_paths = get_paths()
    docs = []
    for doc_path in doc_paths:
        # read document
        with open(doc_path, 'r', encoding='utf-8') as f:
            doc = f.read()
        doc_name = doc_path.split('/')[-1]
        # split document into paragraphs
        doc_data = fragment_doc(doc)
        # add document name and path to dict
        doc_data['doc_name'] = doc_name
        doc_data['doc_path'] = doc_path
        docs.append(doc_data)
    return docs

In [5]:
def calculate_cost(document_df):
    """Takes the prepared dataframe of documents and calculates metadata for entire database."""
    costs = {'num_docs': len(document_df),
                 'num_paras': document_df['num_paras'].sum(),
                 'num_tokens': document_df['num_tokens'].sum(),
                 'est_cost_usd': np.round((document_df['num_tokens'].sum() * 0.0004 / 1000), 2)}
    return costs

In [6]:
def format_for_embedding(df):
    """
    Explode the paragraphs into separate rows, generate a unique id for each paragraph.
    Drop columns that are no longer needed.
    Return: dataframe with one paragraph per row, and a unique id for each paragraph
    """
    df = df.copy()
    # explode the paragraphs into separate rows, generate a unique id for each paragraph
    df = df.explode('text').reset_index(drop=True)
    df = df.drop(columns=['num_sents', 'num_paras', 'num_tokens'])
    df['id']= [uuid.uuid4().hex for _ in range(len(df))]
    while len(df['id'].unique()) != len(df):
        df['id']= [uuid.uuid4().hex for _ in range(len(df))]
    # calculate tokens for each paragraph
    df['num_tokens'] = df['text'].apply(lambda x: len(enc.encode(x)))
    return df

In [7]:
def get_embedding(text):
    text = text.replace("\n", " ")
    try:
        res = openai.Embedding.create(input = [text], model=EMBEDDING_MODEL)['data'][0]['embedding']
    except:
        res = np.zeros(EMBEDDING_LENGTH)
        print("ERROR: Unable to embed text: ", text) # TODO: propagate error and handle it
        logging.error(f'Unable to embed text: {text}')
    return res

In [8]:
def enrich_texts(df, col_name):
    df = df.copy()
    df['enriched_text'] = df['doc_name'] + ': ' + df[col_name]
    return df

In [9]:
def embed_documents(df, col_name):
    df = df.copy()
    df['embedding'] = df[col_name].progress_apply(get_embedding)
    return df

In [10]:
def process_documents():
    """
    Main function to ingest, embed, and store document data and embeddings.
    """
    logging.debug('process_documents(1/4): Attempting to process document database')
    docs = read_documents()
    df = pd.DataFrame(docs)
    costs = calculate_cost(df)
    print('documents loaded into memory')
    print(f'details: {costs}')
    logging.debug(f'process_documents(2/4): Documents loaded into memory. Details: {costs}')

    print('embedding documents...')
    df = format_for_embedding(df)
    df = enrich_texts(df, 'text')
    df = embed_documents(df, 'enriched_text') # Note: this embeds the enriched text
    logging.debug('process_documents(3/4): Embedding complete')
    # store df as pickle in data directory
    df.to_pickle(os.getcwd() + '/data/embeddings.pkl')
    print('embeddings dataframe saved to data/embeddings.pkl')
    logging.debug('process_documents(4/4): Successfully processed document database, data saved to data/embeddings.pkl')
    return None

TODO - enrich each paragraph with the header and section name
this is context dependent, depends on how the text is written, and how chunking is done
decide whether to embed the enriched text, or just the original text

In [11]:
def nearest_neighbors(query_embedding, df, col_name, token_limit=1500, similarity_threshold=0.75, k=10):
    """
    Find the nearest neighbors for a given embedding vector.
    Uses cosine similarity to find the nearest neighbors.
    Returns: dataframe with the nearest neighbors filtered by token limit, similarity threshold, and k
    """
    df = df.copy()
    # calculate cosine similarity between query embedding and all embeddings in df
    df['similarity'] = df[col_name].apply(lambda x: cosine_similarity([query_embedding], [x])[0][0])
    df = df.sort_values(by='similarity', ascending=False)
    # apply constraints
    constraints = {'k': k, 
                   'token_limit': sum(df.num_tokens.cumsum() < token_limit),
                   'similarity_threshold': sum(df.similarity > similarity_threshold)}
    # find the smallest constraint
    constraint = min(constraints, key=constraints.get)
    # slice df to include rows limited by the smallest constraint
    df = df.iloc[:constraints[constraint]]      
    
    return df, constraint


In [12]:
def engineer_prompt(prompt, context=None):
    """Converts a prompt into openai's message list format."""

    query =  f"""Use the below KNOWLEDGE BASE to answer the subsequent question. If KNOWLEDGE BASE is not relevant to the QUESTION, reply with only three words: "I don't know.".

KNOWLEDGE BASE:
\"\"\"
{context}
\"\"\"

QUESTION: {prompt}"""

    messages = [{"role": "system", "content": "You answer questions using the KNOWLEDGE BASE."},
                {"role": "user", "content": query}]
    return messages

In [13]:
def get_answer(input):
    """Takes a user input and returns the answer from the knowledge base."""
    logging.info(f'get_answer(1/2): received input: {input}')
    input_embedding = get_embedding(input)
    results, constraint = nearest_neighbors(input_embedding, df, 'embedding')

    # if results dataframe is empty, return None
    if results.empty:
        logging.info(f'get_answer(2/2): No results due to constraint:{constraint}')
        print(f'No results found. Please try again with a different input.')
        return None, constraint
    else:
        context = results['enriched_text'].str.cat(sep='\n')
    
    # get answer from model with engineered prompt
    messages = engineer_prompt(input, context)
    try:
        answer = openai.ChatCompletion.create(model=ACTIVE_MODEL, messages=messages, temperature=0)
    except:
        error_code = answer['error']['code']
        print(f'Please try again later. Error code: {error_code} from openai')
        logging.info(f'get_answer(2/2): Unsuccessful. Error code: {error_code} from openai.ChatCompletion.create()')
        return None, constraint
    
    logging.info(f'get_answer(2/2): Successful. Details: {json.dumps(answer)}')
    return answer, constraint

In [14]:
def main():

    # initialize global variables and logging
    PATH_TO_DATABASE = os.getcwd() + '/documents'
    CHUNK_SIZE = 5 # number of sentences per paragraph
    CHUNK_OVERLAP = 1 # number of sentences to overlap between paragraphs 

    EMBEDDING_MODEL = "text-embedding-ada-002"
    EMBEDDING_TOKEN_LIMIT = 8191-2 # account for the 2 tokens added by the model
    EMBEDDING_LENGTH = 1536 # length of the embedding vector
    EMBEDDING_BATCH_SIZE = 25 # number of paras to embed at once

    ACTIVE_MODEL = "gpt-3.5-turbo"
    ACTIVE_TOKEN_LIMIT = 4096-2

    logging.basicConfig(filename=f'{os.getcwd()}/logs/babel.log', 
                                    format='%(asctime)s-%(levelname)s-%(message)s', 
                                    level=logging.INFO)
    with open(os.getcwd() + '/secrets/keys.json') as f:
        keys = json.load(f)
    openai.api_key = keys['OPENAI_API_KEY']
    enc = tiktoken.encoding_for_model(EMBEDDING_MODEL)

    # load embeddings into memory
    if os.path.exists(os.getcwd() + '/data/embeddings.pkl'):
        try: 
            df = pd.read_pickle(os.getcwd() + '/data/embeddings.pkl')
            print('embeddings.pkl loaded into memory')
        except:
            print('embeddings.pkl exists, but could not be loaded into memory')
            logging.critical('embeddings.pkl exists, but could not be loaded into memory')
        costs = calculate_cost(pd.DataFrame(read_documents()))
        print(f'to refresh the database, reprocessing all documents would cost: {costs["est_cost_usd"]} USD')
    else:
        print('embeddings.pkl does not exist, processing documents...')
        process_documents()

    # Q & A loop
    user_input = "How does one drink a dinosaur?"
    answer, constraint = get_answer(user_input)
    if answer:
        print(answer.choices[0]['message']['content'])

In [ ]:
if __name__ == '__main__':
    main()

In [16]:
# initialize global variables and logging
PATH_TO_DATABASE = os.getcwd() + '/documents'
CHUNK_SIZE = 5 # number of sentences per paragraph
CHUNK_OVERLAP = 1 # number of sentences to overlap between paragraphs 

EMBEDDING_MODEL = "text-embedding-ada-002"
EMBEDDING_TOKEN_LIMIT = 8191-2 # account for the 2 tokens added by the model
EMBEDDING_LENGTH = 1536 # length of the embedding vector
EMBEDDING_BATCH_SIZE = 25 # number of paras to embed at once

ACTIVE_MODEL = "gpt-3.5-turbo"
ACTIVE_TOKEN_LIMIT = 4096-2

logging.basicConfig(filename=f'{os.getcwd()}/logs/babel.log', 
                                format='%(asctime)s-%(levelname)s-%(message)s', 
                                level=logging.INFO)
with open(os.getcwd() + '/secrets/keys.json') as f:
    keys = json.load(f)
openai.api_key = keys['OPENAI_API_KEY']
enc = tiktoken.encoding_for_model(EMBEDDING_MODEL)

# load embeddings into memory
if os.path.exists(os.getcwd() + '/data/embeddings.pkl'):
    try: 
        df = pd.read_pickle(os.getcwd() + '/data/embeddings.pkl')
        print('embeddings.pkl loaded into memory')
    except:
        print('embeddings.pkl exists, but could not be loaded into memory')
        logging.critical('embeddings.pkl exists, but could not be loaded into memory')
    costs = calculate_cost(pd.DataFrame(read_documents()))
    print(f'to refresh the database, reprocessing all documents would cost: {costs["est_cost_usd"]} USD')
else:
    print('embeddings.pkl does not exist, processing documents...')
    process_documents()

embeddings.pkl loaded into memory
to refresh the database, reprocessing all documents would cost: 0.03 USD


In [ ]:
input = "How does one drink a dinosaur?"

input_embedding = get_embedding(input)
results, constraint = nearest_neighbors(input_embedding, df, 'embedding')

# if results dataframe is empty, return None
if results.empty:
    logging.info(f'get_answer(2/2): No results due to constraint:{constraint}')
    print(f'No results found. Please try again with a different input.')
    return None, constraint
else:
    context = results['enriched_text'].str.cat(sep='\n')

# get answer from model with engineered prompt
messages = engineer_prompt(input, context)
try:
    answer = openai.ChatCompletion.create(model=ACTIVE_MODEL, messages=messages, temperature=0)
except:
    error_code = answer['error']['code']
    print(f'Please try again later. Error code: {error_code} from openai')
    return None, constraint



In [17]:
input = "How does one drink a dinosaur?"

input_embedding = get_embedding(input)
results, constraint = nearest_neighbors(input_embedding, df, 'embedding')

In [19]:
results.head()

,text,doc_name,doc_path,id,num_tokens,enriched_text,embedding,similarity
60,Milo dinosaur or Milo tabur is a Malaysian/Sin...,Milo dinosaur,C:/Users/sihao/Documents/projects/babel/docume...,2467e556cb4e4acd80f5a43b6d4a7a1e,190,Milo dinosaur: Milo dinosaur or Milo tabur is ...,"[0.00023811885330360383, -0.030089786276221275...",0.828739
63,The name of the beverage is one of the terms t...,Milo dinosaur,C:/Users/sihao/Documents/projects/babel/docume...,3c2fec6aad694c838e9cc75894a23e5c,90,Milo dinosaur: The name of the beverage is one...,"[-0.00988902896642685, -0.031585853546857834, ...",0.827594
62,The amount of powder added can be excessive or...,Milo dinosaur,C:/Users/sihao/Documents/projects/babel/docume...,bbd6564679494a25afe1533ef3a738d6,172,Milo dinosaur: The amount of powder added can ...,"[0.002130742883309722, -0.02586507983505726, -...",0.827039
61,"At present, Malaysia has the world's highest p...",Milo dinosaur,C:/Users/sihao/Documents/projects/babel/docume...,6ba00dd43bbe43de8d9de9b53bc73594,92,"Milo dinosaur: At present, Malaysia has the wo...","[0.013058049604296684, -0.015157836489379406, ...",0.816668
265,"Giles Rose made turtle soup as follows: ""Take ...",Turtle soup,C:/Users/sihao/Documents/projects/babel/docume...,e255d1e5d5404ad78f0bb72a62d06029,270,Turtle soup: Giles Rose made turtle soup as fo...,"[0.036201100796461105, -0.008135446347296238, ...",0.776234


In [21]:
# assemble a string containing all the doc_name and id from results
citations = ''
for row in results.itertuples():
    citations += f'{row.doc_name} - {row.id}\n'


In [22]:
citations

'Milo dinosaur - 2467e556cb4e4acd80f5a43b6d4a7a1e\nMilo dinosaur - 3c2fec6aad694c838e9cc75894a23e5c\nMilo dinosaur - bbd6564679494a25afe1533ef3a738d6\nMilo dinosaur - 6ba00dd43bbe43de8d9de9b53bc73594\nTurtle soup - e255d1e5d5404ad78f0bb72a62d06029\nTeh tarik - dd16f78e28a549e5b2e1d7475d39949d\nDrunken prawn - 3d3264a7a4c84fa38de470e778fdedbd\nTurtle soup - 1340d69bf4874e67b326e984de0e465a\nTurtle soup - 9a315cf7b78849f594c7c5d1424c29eb\n'